# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [2]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests
from geoviews import opts
import sys

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
sys.path.append('../')
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [2]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head(10)

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,0,Camana,PE,75.15,-16.6228,-72.7111,Caja Arequipa
1,6,Avarua,CK,78.85,-21.2078,-159.7750,Paradise Inn
2,10,Kapaa,US,71.92,22.0752,-159.3190,Pono Kai Resort
3,16,Cabo San Lucas,MX,77.50,22.8909,-109.9124,Comfort Rooms
4,20,Puerto Ayora,EC,82.36,-0.7393,-90.3518,Hostal La Mirada De Solitario George
5,25,Sao Joao Da Barra,BR,73.49,-21.6403,-41.0511,Pousada Mediterrâneo
6,26,Ribeira Brava,PT,72.01,32.6500,-17.0667,Cheerfulway Bravamar Hotel
7,27,Kavieng,PG,80.01,-2.5744,150.7967,Kavieng Hotel
8,29,Nouadhibou,MR,78.78,20.9310,-17.0347,Hotel valencia
9,33,Georgetown,MY,78.73,5.4112,100.3354,Page 63 hostel


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [3]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Lng", "Lat", "City", "Max Temp", "Hotel Name", "Country","Current Description"]
)

# Display the map
map_plot



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [4]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"] == "Guaymas"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Guaymas"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Cabo San Lucas"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Lazaro Cardenas"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Tomatlan"]


## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [5]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
mid_itinerary_df = [vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end]
itinerary_df = pd.concat(mid_itinerary_df)
# Display sample data
display(itinerary_df)

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
77,336,Guaymas,MX,70.45,27.9333,-110.9000,Hotel Playa de Cortéz
3,16,Cabo San Lucas,MX,77.50,22.8909,-109.9124,Comfort Rooms
56,234,Lazaro Cardenas,MX,83.82,17.9583,-102.2000,Hotel Sol del Pacífico
122,539,Tomatlan,MX,86.61,19.9333,-105.2500,Hotel Hacienda Vieja Tomatlan
77,336,Guaymas,MX,70.45,27.9333,-110.9000,Hotel Playa de Cortéz


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [6]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["Lat", "Lng"]]
# hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
# Display sample data
waypoints_df.head()

,Lat,Lng
77,27.9333,-110.9000
3,22.8909,-109.9124
56,17.9583,-102.2000
122,19.9333,-105.2500
77,27.9333,-110.9000


## Use GeoViews to create map that shows the four cities in the itinerary

In [7]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

In [8]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [9]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [10]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': 'e438d2033e714702acbb050865b68df4',
 'waypoints': '27.9333,-110.9|22.8909,-109.9124|17.9583,-102.2|19.9333,-105.25|27.9333,-110.9'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [11]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()

#display
display(route_response)

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [-110.9, 27.9333], 'original_index': 0},
     {'location': [-109.9124, 22.8909], 'original_index': 1},
     {'location': [-102.2, 17.9583], 'original_index': 2},
     {'location': [-105.25, 19.9333], 'original_index': 3},
     {'location': [-110.9, 27.9333], 'original_index': 4}],
    'units': 'metric',
    'distance': 8146837,
    'distance_units': 'meters',
    'time': 323098.267,
    'legs': [{'distance': 2298208,
      'time': 94132.617,
      'steps': [{'from_index': 0,
        'to_index': 10,
        'distance': 127,
        'time': 13.062,
        'instruction': {'text': 'Drive south.'}},
       {'from_index': 10,
        'to_index': 15,
        'distance': 288,
        'time': 30.188,
        'instruction': {'text': 'Bear left onto Calle Plutarco Elías Calles.'}},
       {'from_index': 15,
        'to_index': 30,
        'distance': 441,
        'time': 45.709,
        'instructi

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [12]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]['geometry']['coordinates']
display(legs)

[[[-110.904732, 27.932836],
  [-110.904758, 27.932745],
  [-110.904756, 27.932702],
  [-110.904746, 27.932652],
  [-110.904724, 27.932599],
  [-110.904685, 27.932464],
  [-110.90468, 27.932369],
  [-110.904685, 27.932299],
  [-110.904697, 27.932236],
  [-110.904713, 27.932171],
  [-110.904842, 27.931724],
  [-110.904745, 27.93156],
  [-110.90438, 27.930911],
  [-110.904131, 27.93032],
  [-110.903915, 27.929634],
  [-110.903675, 27.929375],
  [-110.903763, 27.929036],
  [-110.903814, 27.928964],
  [-110.903836, 27.9289],
  [-110.903991, 27.928453],
  [-110.904181, 27.92785],
  [-110.904564, 27.92667],
  [-110.904684, 27.926244],
  [-110.904712, 27.92613],
  [-110.904733, 27.92604],
  [-110.904743, 27.925972],
  [-110.90475, 27.925892],
  [-110.904748, 27.92582],
  [-110.904741, 27.925744],
  [-110.904727, 27.925671],
  [-110.904691, 27.925564],
  [-110.904994, 27.925417],
  [-110.905367, 27.925272],
  [-110.905703, 27.925166],
  [-110.906012, 27.92509],
  [-110.906044, 27.925085],
  [-1

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [13]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []
step = []
# Loop through the legs coordinates to fetch the latitude and longitude for each step
# for leg in legs:
    
# #     lat = str(leg[0])
# #     long = str(leg[1])
    
# #     print(f"Latitude: {lat}")
# #     print(f"Longitude: {long}")
#     latitude.append(leg[0])
#     longitude.append(leg[1])
    
# print(len(longitude))
# print(len(latitude))

# print(len(legs))

for index in legs:
    length = len(index)
#     print(length)
    for step in index:
        latitude.append(step[0])
        longitude.append(step[1])

# Determining all the different indices
# x = 0

# for i in range(len(longitude)):
#     x = x + 1
#     step.append(x)



## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [14]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame(list(zip(latitude, longitude)),
                        columns=["Latitudes", "Longitudes"])
                        
route_df

# Add the steps' longitude and latitude from each step as columns to the DataFrame

# Display sample data
route_df

,Latitudes,Longitudes
0,-110.904732,27.932836
1,-110.904758,27.932745
2,-110.904756,27.932702
3,-110.904746,27.932652
4,-110.904724,27.932599
...,...,...
71580,-110.904724,27.932599
71581,-110.904746,27.932652
71582,-110.904756,27.932702
71583,-110.904758,27.932745


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [15]:
# Configure the route path by using the GeoViews' Path function

route_path = gv.Path(route_df).opts(color="Pink", line_width=5)
# route_path = gv.Path(color = "Pink")
display(route_path)


:Path   [Latitudes,Longitudes]

In [16]:
# # Display a composed plot by using the route_map and route_path objects
# route_map = route_df.hvplot.points(
#     "Latitudes",
#     "Longitudes",
#     geo = True,
#     tiles = "OSM",
#     frame_width = 700,
#     frame_height = 500,
#     scale = 1,
    
    
# )

# route_map

route_map = waypoints_map * route_path
route_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)
   .Path.I   :Path   [Latitudes,Longitudes]